In [0]:
%run ../functions/utils_silver

In [0]:
table_name="milbom_bronze.bakehouse_sales_transactions"
target_table = "milbom_silver.bakehouse_sales_transactions"
business_keys=["transactionID"]
order_column="_ingestion_timestamp"
partition_column=" "

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS milbom_silver;


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {target_table} (
    transactionID BIGINT PRIMARY KEY COMMENT 'Unique identifier for each sales transaction',
    customerID BIGINT COMMENT 'Unique identifier of the customer',
    franchiseID BIGINT COMMENT 'Unique identifier of the sales point',
    dateTime TIMESTAMP COMMENT 'Date and time when the transaction occurred',
    product STRING COMMENT 'Name of the product sold',
    quantity BIGINT COMMENT 'Number of units sold',
    unitPrice DECIMAL(10,2) COMMENT 'Price per unit',
    totalPrice DECIMAL(10,2) COMMENT 'Total transaction amount',
    paymentMethod STRING COMMENT 'Payment method used',
    _ingestion_timestamp TIMESTAMP COMMENT 'Timestamp of data ingestion',
    _ingestion_date DATE COMMENT 'Date of ingestion'
)
USING DELTA
COMMENT 'Silver layer table containing the latest version of sales transactions';
""")


In [0]:
df_bronze = read_latest_bronze_table(
    table_name=table_name,
    business_keys=business_keys,
    order_column=order_column
)
df_silver = (
    df_bronze
    .select(
        col("transactionID").cast("bigint"),
        col("customerID").cast("bigint"),
        col("franchiseID").cast("bigint"),
        col("dateTime").cast("timestamp"),
        col("product"),
        col("quantity").cast("bigint"),
        col("unitPrice").cast("decimal(10,2)"),
        col("totalPrice").cast("decimal(10,2)"),
        col("paymentMethod"),
        col("_ingestion_timestamp"),
        col("_ingestion_date")
    )
    .dropDuplicates(["transactionID"])
)

merge_condition="target.transactionID = source.transactionID"

write_silver_table(
    df=df_silver,
    target_table=target_table,
    merge_condition=merge_condition,
    optimize=True
)
